<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/wikipedia_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikidata --quiet

In [2]:
import pandas as pd
import requests
from wikidata.client import Client
import time

In [3]:
# Import of the training data
from google.colab import files
uploaded = files.upload()

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv


In [ ]:
# validation data
!pip install datasets --quiet

from datasets import load_dataset
from huggingface_hub import login

login(token="")

dataset = load_dataset('sapienzanlp/nlp2025_hw1_cultural_dataset')
df_dev = dataset['validation'].to_pandas()

Functions to extract the Wikipedia text for a given item

In [10]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

def get_wikipedia_extract(wikidata_url):
    """return the Wikipedia text for a given wikidata url"""

    entity_id = extract_entity_id(wikidata_url)

    client = Client()
    try:
        item = client.get(entity_id, load=True)
        # Get english wikipedia sitelink
        sitelinks = item.data.get("sitelinks", {})
        enwiki = sitelinks.get("enwiki")

        if enwiki:
            title = enwiki["title"]

            #Wikipedia API to get the text
            api_url = "https://en.wikipedia.org/w/api.php"
            params = {
                "action": "query",
                "prop": "extracts",
                "explaintext": True,
                "titles": title,
                "format": "json",
                "redirects": 1
            }

            res = requests.get(api_url, params=params).json()
            page = next(iter(res["query"]["pages"].values()))
            return page.get("extract", "No extract found.")
        else:
            return "No English Wikipedia article found for this entity."
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Entity not found for URL: {wikidata_url}")
            return "Entity not found in Wikidata."
        else:
            raise e  # Re-raise other HTTP errors
    except Exception as e:
        # Catch any other exception that may have occurred during Wikidata entity retrieval
        print(f"An unexpected error occurred for URL: {wikidata_url}. Error: {e}")
        return "Error retrieving Wikidata entity."


def safe_get_wikipedia_extract(url):
    time.sleep(1)  # 1s pause between every request
    return get_wikipedia_extract(url)

a 1 second pause is introduced to prevent wikipedia queries from being blocked

#Getting the text for the training set

In [ ]:
from tqdm import tqdm
tqdm.pandas()

df_train = pd.read_csv('/content/[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep = '\t')
df_train['text'] = df_train['item'].progress_apply(safe_get_wikipedia_extract)

In [15]:
df_train.to_json('train_with_text.json', orient='records', lines=True)

item without text :

In [58]:
df_train[(df_train['text'] == 'Error retrieving Wikidata entity.') |
          (df_train['text'] == "No English Wikipedia article found for this entity.")]


,item,name,description,type,category,subcategory,label,text
1058,http://www.wikidata.org/entity/Q7551241,social media and television,Emerging platforms,entity,media,television,cultural agnostic,Error retrieving Wikidata entity.
1718,http://www.wikidata.org/entity/Q6394187,Kernel Normal Form,Coding style used in the development of code f...,entity,literature,writing style,cultural exclusive,No English Wikipedia article found for this en...


#Application on the dev set

In [29]:
df_dev['text'] = df_dev['item'].progress_apply(safe_get_wikipedia_extract)

100%|██████████| 300/300 [13:05<00:00,  2.62s/it]


In [32]:
df_dev.to_json('dev_with_text.json', orient='records', lines=True)

In [61]:
print(df_dev['text'][0][:500])

1. Fußball-Club Nürnberg Verein für Leibesübungen e. V., often called 1. FC Nürnberg (German pronunciation: [ɛfˌtseː ˈnʏʁnbɛʁk], English: 1. Football Club Nuremberg), is a German sports club based in Nuremberg, Bavaria. It is best known for its men's football team, who currently compete in the 2. Bundesliga. Founded in 1900, the club initially competed in the Southern German championship, winning their first title in 1916. Their first German championship was won in 1920. Before the inauguration 


In [59]:
df_dev[(df_dev['text'] == 'Error retrieving Wikidata entity.') |
          (df_dev['text'] == "No English Wikipedia article found for this entity.")]

,item,name,description,type,category,subcategory,label,text
